## Goal: 
1. get more familiar with huggingface
2. Try a few prompt engineering

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import torch


# Check if GPU is available and set the device
device = 0 if torch.cuda.is_available() else -1

generator = pipeline("text-generation", model="meta-llama/Llama-3.1-8B", device = device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import datasets

In [15]:
def generate_grading_feedback(prompt_template, rubric_text, assignment_text, 
                              max_new_tokens=300, temperature=0.7, top_p=0.9, num_return_sequences=1):

    # Fill in the prompt template with the rubric, assignment, and task description
    prompt = prompt_template.format(
        rubric=rubric_text, 
        assignment=assignment_text, 
    )

    # Generate the output using the given parameters
    outputs = generator(
        prompt, 
        max_new_tokens=max_new_tokens, 
        temperature=temperature, 
        top_p=top_p, 
        num_return_sequences=num_return_sequences,
        pad_token_id=128001
    )
    
    # Return generated text
    return [output['generated_text'] for output in outputs]



In [16]:
prompt_temp = """
You are a helpful grading assistant. 

Task: Generate a grade for the assignment base on the rubric and a structured feedback for both the teacher and students.
Please read the content of the following two documents.

Rubric:
{rubric}

Student Assignment:
{assignment}

There is going to be two main task for you, 
1, grade the assignment base on the rubric, 
 - if the rubric has grade, grade the assignment base on that grade level.
 - show reason for each points off with a short sentence or just a few words.
 - Create a table for the grading in csv format.
 
2. Write a few personalize feedback for the students
"""
prompt_list = [prompt_temp]

In [17]:
## Load the rubric and assignment
rubric_test_path = '/Users/uumin/Documents/QMSS/f24/Grademate/ym_GM/data/train/rb1/10th_Grade_Literature.md'
assignment_test_path = '/Users/uumin/Documents/QMSS/f24/Grademate/ym_GM/data/train/rb1/rb1_asi/a1.txt'

with open(rubric_test_path, 'r') as file:
    rubric_test = file.read()

with open(assignment_test_path, 'r') as file:
    assignment_test = file.read()


In [ ]:
# Generate feedback with specified prompt engineering
feedback = generate_grading_feedback(
    prompt_template=prompt_temp,
    rubric_text=rubric_test, 
    assignment_text=assignment_test, 
    max_new_tokens=300, 
    temperature=0.6,  # Lower temperature for deterministic grading
    top_p=0.9, 
    num_return_sequences=1
)

# Output the generated feedback
print(feedback[0])


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


## Task: Generate a grade for the assignment base on the rubric and a structured feedback for both the teacher and students.

**Initial Prompt**:
"
There is going to be two main task for you, 
1, grade the assignment base on the rubric, 
 - if the rubric has grade, grade the assignment base on that grade level.
 - show reason for each points off with a short sentence or just a few words.
 - Create a table for the grading.
2. Write a few personalize feedback for the students"

Base Prompt